In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import cv2
import numpy as np
import pandas as pd

In [2]:
tf.logging.set_verbosity('ERROR')
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/1", trainable=True);
height, width = hub.get_expected_image_size(module);
shape = (height, width)

In [3]:
# tf.global_variables()
# module.variable_map
LOG_DIR = '.graph'

In [4]:
# resize images in train folder if not in shape
def resize(shape=shape):
    
    originalFolder = 'train'
    resizedFolder  = 'resTrain'
    
    if not os.path.exists(resizedFolder):
        os.makedirs(resizedFolder)
        counter = 0
        # Iterate through resizing and saving
        for img in os.listdir('train'):
            counter = counter + 1
            pic = cv2.imread("train/" + img, cv2.IMREAD_UNCHANGED)
            pic = cv2.resize(pic, shape)
            cv2.imwrite(resizedFolder + '/' + img, pic)
            print("resized image : " + str(shape) + str(counter) + " : "+img + " : " );
    else: 
        print("folder already exists");
        return

# resize images to resnet accepted shape
resize(shape)

folder already exists


In [5]:
label = pd.read_csv("labels.csv", delimiter=',', index_col=0)
labelOneHot = pd.get_dummies(label.breed)
global curr_batch
curr_batch = 0

'''
input batch-size, default = 100
uses global variable curr_batch to remember current batch location
returns tuple (list of labels, 4-d array->(batch, h, w, c))
'''
def next_batch(batch_size=100):
    global curr_batch
    next_batch = True
    start = curr_batch*batch_size
    end   = (curr_batch+1)*batch_size
    print("feeding Batch : " + str(curr_batch))
    # Check is it's last batch
    if ((curr_batch+1)*batch_size>len(label)):
        end = len(file_list)-1
        next_batch = False
    
    batch_list = label.index[start:end]
    label_list = labelOneHot[start:end]
    batch = []
    for img in batch_list: 
        pic = cv2.imread("resTrain/"+img+".jpg", cv2.IMREAD_UNCHANGED)
        batch.append(pic)
    batch = np.stack(batch, axis=0)
    curr_batch = curr_batch + 1

    
    return next_batch, batch_list, label_list, batch


In [6]:
with tf.name_scope("retrain_layer"):
    images = tf.placeholder(dtype=tf.float32, shape=(None, height, width, 3), name="input_images")
    y      = tf.placeholder(dtype=tf.int32, shape=(None, 120), name="labels")

    features = module(images)  # Features with shape [batch_size, 2048], last batch may differ in size
    dense1 = tf.layers.dense(inputs=features, units=1024, activation=tf.nn.relu, name="dense_1")
    logits = tf.layers.dense(inputs=dense1, units=120, name="dense_2")
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax")
    }
    xent = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y, logits=logits))

    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01) \
                    .minimize(loss=xent, global_step=tf.train.get_global_step())

    eval_metric = {
        "accuracy": tf.metrics.accuracy(labels=y, predictions=predictions["probabilities"])}

    tf.summary.scalar('cross_entropy', xent)
    tf.summary.scalar('accuracy', eval_metric["accuracy"][0])

    # Set up logging for predictions
    tensors_to_log = {"probabilities":"accuracy"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)

    summ = tf.summary.merge_all()

In [ ]:
batchNext = True
curr_batch = 0
counter = 0

with tf.Session() as sess:
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    saver  = tf.train.Saver()
    sess.run(init_g)
    sess.run(init_l)

    while(batchNext):
        batchNext, imglist, batch_label, image = next_batch(10)  # A batch of images with shape [batch_size, height, width, 3].

        [train_accuracy, s] = sess.run([eval_metric["accuracy"], summ], feed_dict={images: image, y: batch_label})
        writer.add_summary(s, counter)
        print("training accuracy : " + str(train_accuracy[0]))
        if counter % 20 == 0:
    #         sess.run(assignment, feed_dict={x: , y: mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(LOG_DIR, "kaggle-dog-identification.ckpt"), counter)
        sess.run(train_step, feed_dict={images: image, y: batch_label})
        counter = counter+1


feeding Batch : 0
training accuracy : 0.0
feeding Batch : 1
training accuracy : 0.98333335
feeding Batch : 2
training accuracy : 0.49166667
feeding Batch : 3
training accuracy : 0.32777777
feeding Batch : 4
training accuracy : 0.24583334
feeding Batch : 5
training accuracy : 0.19666667
feeding Batch : 6
training accuracy : 0.16388889
feeding Batch : 7
training accuracy : 0.1404762
feeding Batch : 8
training accuracy : 0.12291667
feeding Batch : 9
training accuracy : 0.10925926
feeding Batch : 10
training accuracy : 0.09833334
feeding Batch : 11
training accuracy : 0.089393936
feeding Batch : 12
training accuracy : 0.08194444
feeding Batch : 13
training accuracy : 0.07564103
feeding Batch : 14
training accuracy : 0.0702381
feeding Batch : 15
training accuracy : 0.06555556
feeding Batch : 16
training accuracy : 0.061458334
feeding Batch : 17
training accuracy : 0.057843138
feeding Batch : 18
training accuracy : 0.05462963
feeding Batch : 19
training accuracy : 0.051754385
feeding Batch :

In [ ]:
# xent

In [ ]:
# merged_summary = tf.summary.merge_all()
# writer = tf.summary.FileWriter(LOG_DIR, tf.get_default_graph())

# init_g = tf.global_variables_initializer()
# init_l = tf.local_variables_initializer()
# sess = tf.Session()
# sess.run(init_g)
# sess.run(init_l)
# sess.run(optimizer)